# Scripts for daily routines (e.g. splitting FASTA files, concatenating files ...)

In [1]:
fasta_file = '../data/curvibacter_sequences/curvibacter_aep_eps_operon.faa'

In [8]:
def split_fasta_file_into_multiple_files(filename,destination):
    try:
        i = open(filename,'r')
        lines = i.readlines()
        i.close()
        
        o = None
        for line in lines:
            if ">" in line:
                if o != None:
                    o.close()
                o = open(destination + '/' + ''.join(line.split('>')[1]).split(' ')[0]+'.faa','w')
            o.write(line)
        
        o.close()
    except Exception as e:
        print("Exception occured : {}".format(e))
        raise Exception('couldnt transform fasta file into multiple files')

In [9]:
split_fasta_file_into_multiple_files(fasta_file,'../data/curvibacter_sequences/eps_operon')

## regular expressions

In [1]:
import re

In [2]:
regex = re.compile('([A,C,G,T]{10,20}TTT[A,C,G])')

In [3]:
genome1_file = open('../data/motif_finding/GCF_000005845.2_ASM584v2_cds_from_genomic.faa','r')
lines1 = genome1_file.readlines()
genome1_file.close()

In [4]:
genome2_file = open('../data/motif_finding/GCF_000009725.1_ASM972v1_cds_from_genomic.faa','r')
lines2 = genome2_file.readlines()
genome2_file.close()

In [11]:
genome1_dict = {}
for line in lines1:
    if '>' in line:
        key = line.split(' ')[0].split('cds_')[-1]
        genome1_dict[key] = ''
    else:
        line = line.strip()
        genome1_dict[key] += line

In [14]:
genome2_dict = {}
for line in lines2:
    if '>' in line:
        key = line.split(' ')[0].split('cds_')[-1]
        genome2_dict[key] = ''
    else:
        line = line.strip()
        genome2_dict[key] += line

In [16]:
result_dict_genome1 = {}
for key in genome1_dict.keys():
    result_dict_genome1[key] = re.findall(regex, genome1_dict[key])

In [17]:
result_dict_genome2 = {}
for key in genome2_dict.keys():
    result_dict_genome2[key] = re.findall(regex, genome2_dict[key])

In [18]:
for key in result_dict_genome1.keys():
    for result in result_dict_genome1[key]:
        for key2 in result_dict_genome2.keys():
            if str(result) in result_dict_genome2[key2]:
                print(key,key2)

NP_415105.1_562 WP_010873414.1_2279


In [20]:
result_dict_genome2['WP_010873414.1_2279']

['ATGATTAAGCAATTTTTC',
 'CCGTTATTGGCTTTA',
 'ATCTAACGCCAACGGTGGGTTTTG',
 'TCAGCGGGAGATAAACAATCTTTG',
 'ATCGACCATAGCTTGAGCTATTTA',
 'TAACCCGTTCCCTCCAGCGATTTA',
 'AGACCGCTGTTGCCAGGGAATTTC',
 'GATGGCAAGGGCACAGTCCATTTC',
 'ACTGGTTATTTTGAACCAGTTTA',
 'TGCCGAATATCGTTATCCCATTTA',
 'TGGTTTGCCCAATAATTTTTC',
 'GACTTGGAAGGTATGGATGGTTTA',
 'GAGGCCAAAGTTCCGTGGCCTTTG',
 'ACGGATTATCCCTACGTAAGTTTA',
 'GAAACCATCCAAGAGGGAATTTTC',
 'CAGCCTGGGGAGGTTACTTTG',
 'CCTAAGTTAATCGACTATTTTG',
 'ATCCCAGCGAACTAAGTCGTTTTA',
 'CCCCGCAATAATCGTCTAATTTTC',
 'CCTCCAGGGGCGATCGCCGTTTTA',
 'AAAAAGTCAACGTTAGTCGTTTTG',
 'GGCCCCGGTCGGGTGGATATTTTC',
 'ACTCCTGGCGATCGAGCCGGTTTG',
 'ATGAGTGACGACGGCCAAATTTA']

In [21]:
for i in result_dict_genome1['NP_415105.1_562']:
    if i in result_dict_genome2['WP_010873414.1_2279']:
        print(i)

CCGTTATTGGCTTTA
